Voici une analyse détaillée de l'intérêt de chaque feature pour prédire la survie dans un cancer du sang :

## **ID (Identifiant patient)**
- **Intérêt direct** : Aucun pour la prédiction (simple identifiant)
- **Utilité** : Essentiel pour l'agrégation des données et le suivi longitudinal
- **Traitement** : À utiliser comme clé de regroupement, pas comme feature prédictive

## **CHR, START, END (Position génomique)** (pas plus important)
- **Intérêt modéré** : Indirect
- **Pourquoi** : Certaines régions chromosomiques sont plus critiques (ex: régions de gènes suppresseurs de tumeurs, oncogènes)
- **Exploitation** : 
  - Peut révéler des "hotspots" mutationnels associés à un mauvais pronostic
  - Utile pour identifier des clusters de mutations
  - Mieux utilisé via des features dérivées (ex: région régulatrice, densité de mutations)

## **REF et ALT (Nucléotides)** ( pas le plus important )
- **Intérêt faible** : Seuls, peu informatifs
- **Pourquoi** : Le type de substitution (transition vs transversion) peut indiquer certains processus mutationnels
- **Exploitation** :
  - Créer des features dérivées : type de mutation (C>T, A>G, etc.)
  - Signature mutationnelle (certains profils sont pronostiques)

## **GENE (Gène affecté)** ⭐ **TRÈS IMPORTANT**
- **Intérêt majeur** : Critical pour le pronostic
- **Pourquoi** : 
  - Certains gènes sont des **marqueurs pronostiques établis** (ex: TP53, FLT3, NPM1, DNMT3A dans les leucémies)
  - La présence/absence de mutations dans des gènes clés est souvent incluse dans les classifications de risque clinique
- **Exploitation** :
  - One-hot encoding ou embedding des gènes fréquents **(fait encoding spécial)**
  - Créer des features : nombre de gènes mutés **fait** , présence de gènes à haut risque **inclus dans l'encoding**
  - Grouper par voies biologiques **(fait)**

## **PROTEIN_CHANGE (Changement protéique)** ⭐ **IMPORTANT**
- **Intérêt élevé** : Très informatif
- **Pourquoi** :
  - Certaines mutations spécifiques sont des "drivers" connus (ex: FLT3-ITD, JAK2 V617F)
  - La position de la mutation dans la protéine affecte sa sévérité (domaine fonctionnel vs non-fonctionnel)
- **Exploitation** :
  - Identifier les mutations récurrentes ("hotspot mutations")
  - Distinguer les mutations dans des domaines critiques
  - Utiliser des bases de données comme COSMIC pour annoter les mutations connues

## **EFFECT (Catégorie d'effet)** ⭐ **TRÈS IMPORTANT**
- **Intérêt majeur** : Excellent prédicteur
- **Pourquoi** :
  - Les mutations avec perte de fonction (frameshift, nonsense) ont souvent un impact plus sévère que les missense
  - Permet de stratifier rapidement l'impact fonctionnel
- **Catégories typiques** : 
  - **Haut impact** : frameshift, stop gained/lost, splice site
  - **Moyen** : missense, inframe indels
  - **Faible** : synonymous, UTR
- **Exploitation** :
  - Feature catégorielle directe
  - Créer des scores agrégés : proportion de mutations à haut impact par patient

## **VAF (Variant Allele Fraction)** ⭐ **EXTRÊMEMENT IMPORTANT**
- **Intérêt critique** : Potentiellement le plus prédictif
- **Pourquoi** :
  - **Charge tumorale** : VAF élevé = proportion importante de cellules mutées = tumeur plus agressive ou plus avancée
  - **Clonalité** : VAF élevé suggère une mutation "fondatrice" (précoce, présente dans toutes les cellules tumorales)
  - **Hétérogénéité** : Distribution des VAF indique la complexité clonale (mauvais pronostic si élevée)
- **Exploitation** :
  - VAF moyen/médian par patient
  - VAF max (mutation dominante)
  - Nombre de mutations avec VAF > seuil (ex: 0.3, 0.5)
  - Distribution/variance des VAF (hétérogénéité)
  - VAF pondéré par l'importance du gène

---

## **Features dérivées recommandées**

Pour maximiser la performance de votre modèle ML, créez ces features agrégées **par patient** :

1. **Charge mutationnelle**
   - Nombre total de mutations somatiques
   - Nombre de mutations à haut impact

2. **Profil génétique**
   - Présence/absence de gènes pronostiques connus
   - Score de risque génétique composite

3. **Métriques de VAF**
   - VAF moyen, médian, max
   - Écart-type des VAF (hétérogénéité)
   - Nombre de mutations avec VAF > 0.4 (mutations clonales)

4. **Complexité tumorale**
   - Nombre de voies biologiques affectées
   - Diversité des effets (entropie des catégories EFFECT)

5. **Interactions génétiques**
   - Co-occurrence de mutations (certaines combinaisons sont synergiques)

Ces données génomiques sont extrêmement riches pour la prédiction de survie dans les cancers hématologiques !

Voici une analyse détaillée de l'intérêt de chaque feature clinique pour prédire la survie :

## **ID (Identifiant patient)**
- **Intérêt** : Aucun (clé de jointure avec les données moléculaires)

## **CENTER (Centre clinique)** 
- **Intérêt modéré** : Potentiellement informatif
- **Pourquoi** :
  - **Qualité des soins** : Variations entre centres (expertise, protocoles)
  - **Biais de sélection** : Certains centres peuvent recruter des cas plus complexes
  - **Facteurs socio-économiques** : Accès aux traitements, suivi
- **Exploitation** :
  - Encoding catégoriel ou groupement par taille/expertise
  - Peut servir de variable de stratification
- **Attention** : Risque de sur-apprentissage si peu de patients par centre

## **BM_BLAST (Blastes médullaires en %)** ⭐ **EXTRÊMEMENT IMPORTANT**
- **Intérêt critique** : Un des marqueurs pronostiques les plus puissants
- **Pourquoi** :
  - **Critère diagnostique** : Définit le type et le stade de la maladie (ex: MDS vs AML si >20%)
  - **Charge tumorale** : % élevé = maladie plus agressive
  - **Classification WHO** : Seuils pronostiques établis (5%, 10%, 20%)
  - Corrélé directement à la survie : plus le % est élevé, pire le pronostic
- **Exploitation** :
  - Feature continue très prédictive
  - Créer des catégories selon classifications cliniques
  - Interaction avec cytogénétique (blast + mauvaise cyto = très mauvais pronostic)

## **WBC (Globules blancs en G/L)** ⭐ **TRÈS IMPORTANT**
- **Intérêt majeur** : Marqueur pronostique établi
- **Pourquoi** :
  - **WBC élevé** (>20-30 G/L) = souvent mauvais pronostic dans les leucémies
  - Reflète la **prolifération tumorale**
  - **Leucocytose** peut indiquer une maladie agressive (ex: leucémie myélomonocytaire chronique)
  - Risque de complications (leucostase, syndrome de lyse tumorale)
- **Exploitation** :
  - Feature continue
  - Transformation log possible (distribution souvent asymétrique)
  - Seuils cliniques : <4, 4-10, 10-30, >30 G/L

## **ANC (Neutrophiles absolus en G/L)** ⭐ **TRÈS IMPORTANT**
- **Intérêt majeur** : Indicateur de fonction médullaire
- **Pourquoi** :
  - **Neutropénie** (<1.5 G/L) = risque infectieux, réserve médullaire faible
  - Reflète la capacité de la moelle à produire des cellules normales
  - ANC bas = moelle "envahie" par les cellules tumorales
  - **Pronostic** : ANC bas corrélé à survie réduite
- **Exploitation** :
  - Feature continue
  - Catégories cliniques : sévère (<0.5), modérée (0.5-1.0), normale (>1.5)

## **MONOCYTES (Monocytes en G/L)** ⭐ **IMPORTANT** (absent dans le test donc drop)
- **Intérêt élevé** : Spécifique selon le type de cancer
- **Pourquoi** :
  - **Monocytose** (>1 G/L) : critère diagnostique de certains cancers (LMMC = leucémie myélomonocytaire chronique)
  - Marqueur de sous-type de maladie
  - Peut indiquer une différenciation monocytaire anormale
- **Exploitation** :
  - Feature continue
  - Ratio monocytes/lymphocytes (si lymphocytes disponibles)
  - Seuil à 1 G/L pour monocytose

## **HB (Hémoglobine en g/dL)** ⭐ **TRÈS IMPORTANT**
- **Intérêt majeur** : Marqueur de sévérité
- **Pourquoi** :
  - **Anémie** = symptôme clé des cancers du sang (insuffisance médullaire)
  - HB bas (<10 g/dL) = mauvais pronostic établi
  - Reflète la capacité de production érythrocytaire
  - Impact sur qualité de vie et performance status
  - Critère majeur dans les scores pronostiques (IPSS, IPSS-R)
- **Exploitation** :
  - Feature continue très prédictive
  - Catégories WHO : sévère (<8), modérée (8-10), légère (10-12)
  - Dépendance aux transfusions (si HB très bas)

## **PLT (Plaquettes en G/L)** ⭐ **EXTRÊMEMENT IMPORTANT**
- **Intérêt critique** : Un des plus puissants prédicteurs
- **Pourquoi** :
  - **Thrombopénie** (<100 G/L) = marqueur pronostique majeur
  - PLT bas = insuffisance médullaire sévère, risque hémorragique
  - **Sévérité** : PLT <50 = mauvais, PLT <20 = très mauvais pronostic
  - Critère central dans tous les scores pronostiques (IPSS, WPSS, IPSS-R)
  - Reflète la fonction mégacaryocytaire
- **Exploitation** :
  - Feature continue extrêmement prédictive
  - Catégories : sévère (<50), modérée (50-100), normale (>150)
  - Transformation log possible

## **CYTOGENETICS (Caryotype)** ⭐ **EXTRÊMEMENT IMPORTANT**
- **Intérêt critique** : LE facteur pronostique le plus puissant dans beaucoup de cancers du sang
- **Pourquoi** :
  - **Classification de risque** : Base des scores pronostiques (IPSS-R)
  - Certaines anomalies sont **très défavorables** : -7/del(7q), -5/del(5q), caryotype complexe (≥3 anomalies), anomalies chromosome 3
  - Certaines sont **favorables** : del(5q) isolée, del(20q), -Y
  - **Caryotype complexe** (≥3 anomalies) = très mauvais pronostic
  - **Caryotype normal** vs **anormal** = différence pronostique majeure
- **Exploitation** :
  - **Parsing ISCN** : Complexe mais essentiel !
    - Identifier : nombre de chromosomes (46 = normal, autres = aneuploidie)
    - Détecter délétions (del, -), additions (+), translocations t()
    - Compter nombre d'anomalies (complexité)
  - **Features à créer** :
    - **Catégorie de risque cytogénétique** : Très bon / Bon / Intermédiaire / Mauvais / Très mauvais (selon classification IPSS-R)
    - Présence d'anomalies spécifiques : -7, del(5q), -5, anomalie 3, etc.
    - **Complexité** : nombre d'anomalies (0, 1, 2, 3+)
    - Caryotype normal (46,XX ou 46,XY) vs anormal
    - Monosomies/trisomies
- **Exemples de classification** :
  - **Très favorable** : -Y isolé, del(11q) isolé
  - **Favorable** : Normal, del(5q) isolé, del(12p) isolé, del(20q) isolé, double incluant del(5q)
  - **Intermédiaire** : del(7q) isolé, +8 isolé, +19 isolé, i(17q) isolé, autres simples/doubles
  - **Défavorable** : -7, inv(3)/t(3q), double incluant -7/del(7q), caryotype complexe (3 anomalies)
  - **Très défavorable** : Caryotype complexe (>3 anomalies)

---

## **Features dérivées recommandées**

### 1. **Scores pronostiques établis** (à recréer)
- **IPSS-R** (International Prognostic Scoring System - Revised) :
  - Combine : BM_BLAST, cytogénétique, HB, PLT, ANC
  - Classification : Très bas / Bas / Intermédiaire / Élevé / Très élevé risque
- **WPSS** (WHO Prognostic Scoring System)

### 2. **Ratios et interactions**
- WBC/ANC (proportion de cellules anormales)
- PLT/HB (sévérité de l'atteinte médullaire)
- BM_BLAST × complexité cytogénétique
- Charge tumorale composite : (BM_BLAST + WBC_log) × risque_cyto

### 3. **Sévérité globale**
- Nombre de cytopénies (HB bas + PLT bas + ANC bas)
- Score composite d'insuffisance médullaire

### 4. **Catégorisation cytogénétique**
- **Essentiel** : Parser le caryotype pour extraire :
  - Risque cytogénétique (selon IPSS-R)
  - Nombre d'anomalies
  - Présence d'anomalies à haut risque (-7, -5, complexe)

---

## **Priorités pour le ML**

**Top 3 features les plus prédictives** :
1. **CYTOGENETICS** (une fois parsé et catégorisé)
2. **BM_BLAST**
3. **PLT**

Ces trois features constituent la base des scores pronostiques cliniques validés et sont probablement les plus importants pour votre modèle.

**Attention** : La cytogénétique nécessite un travail de preprocessing substantiel (parsing ISCN, classification de risque) mais c'est un investissement crucial pour la performance du modèle !

In [2]:
# Import necessary libraries
import pandas as pd

In [3]:
from ens_data_challenge.globals import TRAIN_CLINICAL_DATA_PATH, TRAIN_MOLECULAR_DATA_PATH, TRAIN_TARGET_PATH, TEST_CLINICAL_DATA_PATH, TEST_MOLECULAR_DATA_PATH
clinical_data_train = pd.read_csv(TRAIN_CLINICAL_DATA_PATH)
clinical_data_eval = pd.read_csv(TEST_CLINICAL_DATA_PATH)

# Molecular Data
molecular_data_train = pd.read_csv(TRAIN_MOLECULAR_DATA_PATH)
molecular_data_eval = pd.read_csv(TEST_MOLECULAR_DATA_PATH)

target_df = pd.read_csv(TRAIN_TARGET_PATH)

# Preview the data
clinical_data_train.head()

,ID,CENTER,BM_BLAST,WBC,ANC,MONOCYTES,HB,PLT,CYTOGENETICS
0,P132697,MSK,14.0,2.8,0.2,0.7,7.6,119.0,"46,xy,del(20)(q12)[2]/46,xy[18]"
1,P132698,MSK,1.0,7.4,2.4,0.1,11.6,42.0,"46,xx"
2,P116889,MSK,15.0,3.7,2.1,0.1,14.2,81.0,"46,xy,t(3;3)(q25;q27)[8]/46,xy[12]"
3,P132699,MSK,1.0,3.9,1.9,0.1,8.9,77.0,"46,xy,del(3)(q26q27)[15]/46,xy[5]"
4,P132700,MSK,6.0,128.0,9.7,0.9,11.1,195.0,"46,xx,t(3;9)(p13;q22)[10]/46,xx[10]"


Preprocessiong, dans ce module nous allons utiliser notre helper qui nous permet de recuperer des données plus clean en appliquant quelques imputations selon des heuristiques en créant les données cytogenetique et en ajoutant des infos cytpgénétiques dans le df clinique qui est celui utilisé pour le moment en majorité

In [4]:
from ens_data_challenge.preprocess.preprocessor import Preprocessor

preprocessor = Preprocessor()

In [5]:
clinical_data_train, cyto_df_train = preprocessor.get_cyto_features_and_df(clinical_data_train)
clinical_data_eval, cyto_df_eval = preprocessor.get_cyto_features_and_df(clinical_data_eval)

In [6]:
(
    clinical_preprocess_train,
    clinical_preprocess_eval, 
    molecular_preprocess_train, 
    molecular_preprocess_eval, 
    cyto_struct_preprocess_train, 
    cyto_struct_preprocess_eval,
    targets_preprocess
  ) = preprocessor.fit_transform(
    clinical_data_train=clinical_data_train,
    molecular_data_train=molecular_data_train,
    clinical_data_test=clinical_data_eval,
    molecular_data_test=molecular_data_eval,
    cyto_struct_train=cyto_df_train,
    cyto_struct_test=cyto_df_eval,
    targets=target_df
)

Feature engineering, ici on ajoute des features customs qui suivent les avancées médicales sur le domaine

In [7]:
from ens_data_challenge.feature_engineering.feat_eng_helper import FeatureEngineerHelper
from ens_data_challenge.types import CytoColumns, MolecularColumns, CytoStructColumns, Columns

fe = FeatureEngineerHelper()

In [8]:
clinical_data_train = fe.Nmut(molecular_preprocess_train, clinical_preprocess_train)
clinical_data_eval = fe.Nmut(molecular_preprocess_eval, clinical_preprocess_eval)

In [9]:
clinical_preprocess_train = fe.add_mol_encoding(clinical_preprocess_train, molecular_preprocess_train, col='GENE', method='confidence_weighted', apply_effect_weighting=True)
clinical_preprocess_eval = fe.add_mol_encoding(clinical_preprocess_eval, molecular_preprocess_eval, col='GENE', method='confidence_weighted', apply_effect_weighting=True)

Ajoutées 65 colonnes encodées pour 'GENE' (method=confidence_weighted, apply_effect_weighting=True).
Ajoutées 65 colonnes encodées pour 'GENE' (method=confidence_weighted, apply_effect_weighting=True).


In [10]:
clinical_preprocess_train = fe.add_mol_encoding(clinical_preprocess_train, molecular_preprocess_train, col="PATHWAY", method='confidence_weighted', apply_effect_weighting=True)
clinical_preprocess_eval = fe.add_mol_encoding(clinical_preprocess_eval, molecular_preprocess_eval, col="PATHWAY", method='confidence_weighted', apply_effect_weighting=True)

Ajoutées 8 colonnes encodées pour 'PATHWAY' (method=confidence_weighted, apply_effect_weighting=True).
Ajoutées 8 colonnes encodées pour 'PATHWAY' (method=confidence_weighted, apply_effect_weighting=True).


In [11]:
clinical_preprocess_train = fe.severity_scores(clinical_preprocess_train)
clinical_preprocess_eval = fe.severity_scores(clinical_preprocess_eval)

In [12]:
clinical_preprocess_train = fe.ratios_and_interactions(clinical_preprocess_train)
clinical_preprocess_eval = fe.ratios_and_interactions(clinical_preprocess_eval)

In [13]:
clinical_preprocess_train

,ID,CENTER,BM_BLAST,WBC,ANC,HB,PLT,is_normal,ploidy,has_tp53_deletion,...,Épigénétique__confidence_weighted__effect,Suppresseurs de tumeurs / Réparation de l'ADN__confidence_weighted__effect,Complexe de cohésine__confidence_weighted__effect,Épissage__confidence_weighted__effect,OTHER__confidence_weighted__effect_y,cytopenias_count,wbc_anc_ratio,plt_hb_ratio,blast_cyto_complexity,tumor_burden_composite
0,P132697,MSK,14.0,2.80,0.20,7.6,119.0,False,46,False,...,0.903369,0.0000,0.000000,0.315000,0.0,2,14.000000,15.657895,14.0,2.836975
1,P132698,MSK,1.0,7.40,2.40,11.6,42.0,True,46,False,...,0.539188,0.0000,0.000000,0.000000,0.0,1,3.083333,3.620690,0.0,0.000000
2,P116889,MSK,15.0,3.70,2.10,14.2,81.0,False,46,False,...,0.052500,0.0000,0.000000,0.043200,0.0,1,1.761905,5.704225,15.0,5.559981
3,P132699,MSK,1.0,3.90,1.90,8.9,77.0,False,46,False,...,0.622273,0.0000,0.299934,0.014711,0.0,2,2.052632,8.651685,1.0,0.479009
4,P132700,MSK,6.0,128.00,9.70,11.1,195.0,False,46,False,...,0.224248,0.0000,0.000000,0.000000,0.0,0,13.195876,17.567568,6.0,2.009065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3168,P121826,VU,1.0,2.50,1.02,10.2,78.0,True,46,False,...,0.470434,0.0000,0.000000,0.000000,0.0,2,2.450980,7.647059,0.0,0.000000
3169,P121827,VU,1.5,8.10,2.66,11.3,40.0,False,44,False,...,0.000000,0.2457,0.000000,0.000000,0.0,1,3.045113,3.539823,6.0,1.928303
3170,P121830,VU,0.0,1.80,0.55,9.4,86.0,False,45,False,...,0.107100,0.2335,0.000000,0.322909,0.0,3,3.272727,9.148936,0.0,0.393315
3171,P121853,VU,5.0,1.37,0.37,11.4,102.0,False,46,False,...,0.472500,0.1647,0.000000,0.000000,0.0,1,3.702703,8.947368,5.0,1.084635


In [14]:
risk_cols = [
CytoColumns.COMPUTED_RISK_SCORE.value,
CytoColumns.MDS_IPSS_R_CYTO_RISK.value,
CytoColumns.AML_ELN_2022_CYTO_RISK.value,
CytoColumns.CLL_CYTO_RISK.value,
CytoColumns.MM_RISS_CYTO_RISK.value,
]

In [15]:
clinical_preprocess_eval

,ID,CENTER,BM_BLAST,WBC,ANC,MONOCYTES,HB,PLT,CYTOGENETICS,is_normal,...,Transcription / Différenciation__confidence_weighted__effect,Épigénétique__confidence_weighted__effect,OTHER__confidence_weighted__effect_y,Épissage__confidence_weighted__effect,Suppresseurs de tumeurs / Réparation de l'ADN__confidence_weighted__effect,cytopenias_count,wbc_anc_ratio,plt_hb_ratio,blast_cyto_complexity,tumor_burden_composite
0,KYW1,KYW,68.0,3.45,0.5865,0.0,7.6,48.0,"47,XY,+X,del(9)(q?)[15]/47,XY,+X[5]",False,...,0.000,0.345599,0.0,0.000000,0.036801,3,5.882353,6.315789,68.0,12.856187
1,KYW2,KYW,35.0,3.18,1.2402,0.0,10.0,32.0,"46,XY,der(3)?t(3;11)(q26.2;q23),add(4)(p15).de...",False,...,0.000,0.358855,0.0,0.000000,0.000000,2,2.564103,3.200000,175.0,16.393640
2,KYW3,KYW,3.0,12.40,8.6800,0.0,12.3,25.0,"47,XX,+8",False,...,0.000,0.707475,0.0,0.000000,0.000000,1,1.428571,2.032520,3.0,1.035122
3,KYW4,KYW,61.0,5.55,2.0535,0.0,8.0,44.0,Normal,True,...,0.642,0.000000,0.0,0.000000,0.000000,2,2.702703,5.500000,0.0,0.000000
4,KYW5,KYW,2.0,1.21,0.7381,0.0,8.6,27.0,"43,XY,dic(5;17)(q11.2;p11.2),-7,-13,-20,-22,+r...",False,...,0.000,0.052500,0.0,0.314068,0.000000,3,1.639344,3.139535,8.0,1.312706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188,KYW1189,KYW,3.0,4.10,2.0000,0.0,9.7,123.0,0,True,...,0.000,0.000000,0.0,0.000000,0.000000,1,2.050000,12.680412,0.0,0.000000
1189,KYW1190,KYW,3.0,4.10,2.0000,0.0,9.7,123.0,0,True,...,0.000,0.307800,0.0,0.000000,0.000000,1,2.050000,12.680412,0.0,0.000000
1190,KYW1191,KYW,3.0,4.10,2.0000,0.0,9.7,123.0,0,True,...,0.000,0.000000,0.0,0.000000,0.109800,1,2.050000,12.680412,0.0,0.000000
1191,KYW1192,KYW,3.0,4.10,2.0000,0.0,9.7,123.0,0,True,...,0.000,0.185860,0.0,0.000000,0.000000,1,2.050000,12.680412,0.0,0.000000


In [16]:
drop_columns = [
    Columns.CENTER.value,
    Columns.MONOCYTES.value,
    Columns.CYTOGENETICS.value,
    "has_large_deletion",
    "ID"
]

In [17]:
targets_preprocess.head()

,ID,OS_YEARS,OS_STATUS
0,P132697,1.115068,1.0
1,P132698,4.928767,0.0
2,P116889,2.043836,0.0
3,P132699,2.476712,1.0
4,P132700,3.145205,0.0


In [18]:
event_target = targets_preprocess["OS_STATUS"]
time_target = targets_preprocess["OS_YEARS"]

In [19]:
valid_time_idx = event_target.loc[event_target == 1].index
time_target = time_target.loc[valid_time_idx]

In [20]:
event_target.head()

0    1.0
1    0.0
2    0.0
3    1.0
4    0.0
Name: OS_STATUS, dtype: float64

In [21]:
time_target.describe()

count    1600.000000
mean        2.033873
std         2.061487
min         0.002740
25%         0.669863
50%         1.380822
75%         2.670548
max        17.375342
Name: OS_YEARS, dtype: float64

In [22]:
clinical_preprocess_train

,ID,CENTER,BM_BLAST,WBC,ANC,HB,PLT,is_normal,ploidy,has_tp53_deletion,...,Épigénétique__confidence_weighted__effect,Suppresseurs de tumeurs / Réparation de l'ADN__confidence_weighted__effect,Complexe de cohésine__confidence_weighted__effect,Épissage__confidence_weighted__effect,OTHER__confidence_weighted__effect_y,cytopenias_count,wbc_anc_ratio,plt_hb_ratio,blast_cyto_complexity,tumor_burden_composite
0,P132697,MSK,14.0,2.80,0.20,7.6,119.0,False,46,False,...,0.903369,0.0000,0.000000,0.315000,0.0,2,14.000000,15.657895,14.0,2.836975
1,P132698,MSK,1.0,7.40,2.40,11.6,42.0,True,46,False,...,0.539188,0.0000,0.000000,0.000000,0.0,1,3.083333,3.620690,0.0,0.000000
2,P116889,MSK,15.0,3.70,2.10,14.2,81.0,False,46,False,...,0.052500,0.0000,0.000000,0.043200,0.0,1,1.761905,5.704225,15.0,5.559981
3,P132699,MSK,1.0,3.90,1.90,8.9,77.0,False,46,False,...,0.622273,0.0000,0.299934,0.014711,0.0,2,2.052632,8.651685,1.0,0.479009
4,P132700,MSK,6.0,128.00,9.70,11.1,195.0,False,46,False,...,0.224248,0.0000,0.000000,0.000000,0.0,0,13.195876,17.567568,6.0,2.009065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3168,P121826,VU,1.0,2.50,1.02,10.2,78.0,True,46,False,...,0.470434,0.0000,0.000000,0.000000,0.0,2,2.450980,7.647059,0.0,0.000000
3169,P121827,VU,1.5,8.10,2.66,11.3,40.0,False,44,False,...,0.000000,0.2457,0.000000,0.000000,0.0,1,3.045113,3.539823,6.0,1.928303
3170,P121830,VU,0.0,1.80,0.55,9.4,86.0,False,45,False,...,0.107100,0.2335,0.000000,0.322909,0.0,3,3.272727,9.148936,0.0,0.393315
3171,P121853,VU,5.0,1.37,0.37,11.4,102.0,False,46,False,...,0.472500,0.1647,0.000000,0.000000,0.0,1,3.702703,8.947368,5.0,1.084635


In [23]:
molecular_preprocess_train

,ID,CHR,START,END,REF,ALT,GENE,PROTEIN_CHANGE,EFFECT,VAF,DEPTH
0,P100000,11,119149248.0,119149248.0,G,A,CBL,p.C419Y,non_synonymous_codon,0.08300,1308.0
1,P100000,5,131822301.0,131822301.0,G,T,IRF1,p.Y164*,stop_gained,0.02200,532.0
2,P100000,3,77694060.0,77694060.0,G,C,OTHER,p.?,OTHER,0.41000,876.0
3,P100000,4,106164917.0,106164917.0,G,T,TET2,p.R1262L,non_synonymous_codon,0.43000,826.0
4,P100000,2,25468147.0,25468163.0,OTHER,A,DNMT3A,p.E505fs*141,frameshift_variant,0.08980,942.0
...,...,...,...,...,...,...,...,...,...,...,...
10540,P131472,OTHER,0.0,0.0,OTHER,OTHER,MLL,MLL_PTD,PTD,0.32125,975.0
10541,P131505,OTHER,0.0,0.0,OTHER,OTHER,MLL,MLL_PTD,PTD,0.32125,975.0
10542,P131816,OTHER,0.0,0.0,OTHER,OTHER,MLL,MLL_PTD,PTD,0.32125,975.0
10543,P132717,OTHER,0.0,0.0,OTHER,OTHER,MLL,MLL_PTD,PTD,0.32125,975.0


In [24]:
cyto_struct_preprocess_train

,ID,ploidy,sex_chromosomes,clone_index,clone_cell_count,mutation_type,chromosome,arm,start,end,start_arm,end_arm,raw
0,P132697,46,XY,0,2,deletion,20,q,12,0,q,UNKNOWN,"{'type': 'deletion', 'chromosome': '20', 'arm'..."
1,P116889,46,XY,0,8,translocation,"3,3",UNKNOWN,0,0,q,q,"{'type': 'translocation', 'chromosomes': ('3',..."
2,P132699,46,XY,0,15,deletion,3,q,26,27,q,q,"{'type': 'deletion', 'chromosome': '3', 'arm':..."
3,P132700,46,XX,0,10,translocation,"3,9",UNKNOWN,0,0,p,q,"{'type': 'translocation', 'chromosomes': ('3',..."
4,P132704,45,XX,0,2,deletion,5,q,13,33,q,q,"{'type': 'deletion', 'chromosome': '5', 'arm':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2753,P121830,46,XY,0,4,deletion,20,q,11.2,13.1,q,q,"{'type': 'deletion', 'chromosome': '20', 'arm'..."
2754,P121830,46,XY,1,16,monosomy,7,UNKNOWN,0,0,UNKNOWN,UNKNOWN,7
2755,P121853,46,XX,0,5,deletion,1,p,34,0,p,UNKNOWN,"{'type': 'deletion', 'chromosome': '1', 'arm':..."
2756,P121853,46,XX,1,12,monosomy,18,UNKNOWN,0,0,UNKNOWN,UNKNOWN,18


In [25]:
import numpy as np
numeric_df = clinical_preprocess_train.select_dtypes(include=[np.number])
cols_with_inf = numeric_df[np.isinf(numeric_df).any(axis=1)].columns
print(cols_with_inf)

clinical_preprocess_train = clinical_preprocess_train.replace([np.inf, -np.inf], 0)

numeric_df = clinical_preprocess_train.select_dtypes(include=[np.number])
cols_with_inf = numeric_df[np.isinf(numeric_df).any(axis=1)].columns
print(cols_with_inf)


Index(['BM_BLAST', 'WBC', 'ANC', 'HB', 'PLT', 'ploidy', 'n_abnormalities',
       'n_chromosomes_affected', 'n_deletions', 'n_critical_regions_deleted',
       'n_clones', 'abnormal_clone_percentage', 'computed_risk_score',
       'mds_ipss_r_cyto_risk', 'aml_eln_2022_cyto_risk', 'cll_cyto_risk',
       'mm_riss_cyto_risk', 'CBL__confidence_weighted__effect',
       'IRF1__confidence_weighted__effect',
       'OTHER__confidence_weighted__effect_x',
       'TET2__confidence_weighted__effect',
       'DNMT3A__confidence_weighted__effect',
       'CHEK2__confidence_weighted__effect',
       'TP53__confidence_weighted__effect',
       'STAG2__confidence_weighted__effect',
       'ETNK1__confidence_weighted__effect',
       'JAK2__confidence_weighted__effect',
       'SRSF2__confidence_weighted__effect',
       'EZH2__confidence_weighted__effect',
       'SF3B1__confidence_weighted__effect',
       'CSF3R__confidence_weighted__effect',
       'GATA2__confidence_weighted__effect',
       'CR

In [26]:
clinical_preprocess_train.head()

,ID,CENTER,BM_BLAST,WBC,ANC,HB,PLT,is_normal,ploidy,has_tp53_deletion,...,Épigénétique__confidence_weighted__effect,Suppresseurs de tumeurs / Réparation de l'ADN__confidence_weighted__effect,Complexe de cohésine__confidence_weighted__effect,Épissage__confidence_weighted__effect,OTHER__confidence_weighted__effect_y,cytopenias_count,wbc_anc_ratio,plt_hb_ratio,blast_cyto_complexity,tumor_burden_composite
0,P132697,MSK,14.0,2.8,0.2,7.6,119.0,False,46,False,...,0.903369,0.0,0.000000,0.315000,0.0,2,14.000000,15.657895,14.0,2.836975
1,P132698,MSK,1.0,7.4,2.4,11.6,42.0,True,46,False,...,0.539188,0.0,0.000000,0.000000,0.0,1,3.083333,3.620690,0.0,0.000000
2,P116889,MSK,15.0,3.7,2.1,14.2,81.0,False,46,False,...,0.052500,0.0,0.000000,0.043200,0.0,1,1.761905,5.704225,15.0,5.559981
3,P132699,MSK,1.0,3.9,1.9,8.9,77.0,False,46,False,...,0.622273,0.0,0.299934,0.014711,0.0,2,2.052632,8.651685,1.0,0.479009
4,P132700,MSK,6.0,128.0,9.7,11.1,195.0,False,46,False,...,0.224248,0.0,0.000000,0.000000,0.0,0,13.195876,17.567568,6.0,2.009065


# Modeles entrainement

Reformulaiton du problème, on va commencer par prédire la proba d un event et predire le temps de rechute des patients qui ont déjà été touché, on pourra ensuite construire des features ou une nouvelle formule de score avec ces données. Pour cela on fait un ensemble de plusieurs modeles.

On ajoute des poids sur les entrainements des regresseurs dans la rmse afin de prendre bien en compte les poids de Kaplan meier dans la ipwc, on peut entrainer des regresseurs sur données non censuré sans poids et un regresseur sur toutes les données avec poids.

In [27]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import numpy as np

# TabM import

# Create event_target from 
y = event_target.values

# Prepare X
X = clinical_preprocess_train.drop(columns=drop_columns, errors='ignore').copy()

# Scale for TabM/SVC
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define models
models = [
    ('XGBoost', XGBClassifier(n_jobs=-1, random_state=42, verbosity=0), False),
    ('CatBoost', CatBoostClassifier(verbose=0, random_state=42), False),
    ('LightGBM', LGBMClassifier(n_jobs=-1, random_state=42, verbose=-1), False),
    ('RandomForest', RandomForestClassifier(n_jobs=-1, random_state=42), False),
    ('AdaBoost', AdaBoostClassifier(random_state=42,), False),
    ('KernelSVC', SVC(kernel='rbf', probability=True, random_state=42), True),  # needs scaling
]

# K-Fold CV
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = {}

for name, model, needs_scaling in models:
    print(f"Training {name}...")
    aucs, accs = [], []
    for train_idx, val_idx in kfold.split(X, y):
        if needs_scaling:
            X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
        else:
            X_train, X_val = X.iloc[train_idx].values, X.iloc[val_idx].values
        y_train, y_val = y[train_idx], y[val_idx]
        
        model.fit(X_train, y_train)
        y_proba = model.predict_proba(X_val)[:, 1]
        aucs.append(roc_auc_score(y_val, y_proba))
        accs.append(accuracy_score(y_val, model.predict(X_val)))
    
    results[name] = {'AUC': np.mean(aucs), 'Std': np.std(aucs), 'Acc': np.mean(accs)}
    print(f"  -> {name} AUC: {np.mean(aucs):.4f} (+/- {np.std(aucs):.4f})")

# Results
pd.DataFrame(results).T.sort_values(by='AUC', ascending=False)

Training XGBoost...


  -> XGBoost AUC: 0.6734 (+/- 0.0097)
Training CatBoost...
  -> CatBoost AUC: 0.7140 (+/- 0.0060)
Training LightGBM...


c:\Users\enzo.cAo\Documents\Projects\competitions\ens_data_challenge\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\enzo.cAo\Documents\Projects\competitions\ens_data_challenge\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\enzo.cAo\Documents\Projects\competitions\ens_data_challenge\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\enzo.cAo\Documents\Projects\competitions\ens_data_challenge\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\enzo.cAo\Documents\

  -> LightGBM AUC: 0.6863 (+/- 0.0056)
Training RandomForest...
  -> RandomForest AUC: 0.7002 (+/- 0.0046)
Training AdaBoost...
  -> AdaBoost AUC: 0.7043 (+/- 0.0143)
Training KernelSVC...
  -> KernelSVC AUC: 0.7040 (+/- 0.0165)


,AUC,Std,Acc
CatBoost,0.714040,0.006010,0.662140
AdaBoost,0.704332,0.014263,0.648593
KernelSVC,0.703997,0.016477,0.655214
RandomForest,0.700154,0.004584,0.653321
LightGBM,0.686325,0.005561,0.630946
XGBoost,0.673357,0.009742,0.635986


# Regression

Regression, sur les données qui ont deja connu un déces on va maintenant prédire le moment du deces, cela permet de classer les personnes qui ont déjà eu un probleme, on sépare ainsi un probleme avec information cahcé en deux probleme complet ce qui simplifie la tache

In [28]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from lifelines.utils import concordance_index
import numpy as np
import pandas as pd
from scipy.stats import norm

# Target: OS_YEARS
y_raw = targets_preprocess['OS_YEARS'].values
event = event_target.values

def transform_y(y):
    """Rank -> uniform score -> inverse CDF normale"""
    n = len(y)
    ranks = (-y).argsort().argsort() + 1  # Rank based on -time (higher rank = shorter survival)
    uniform_scores = ranks / (n + 1)  # Avoid 0 and 1 for ppf stability
    return norm.ppf(uniform_scores)  # Inverse CDF normale

def inverse_transform_pred(pred):
    """CDF normale -> uniform scores (for ranking comparison)"""
    return norm.cdf(pred)

# Prepare X
X = clinical_preprocess_train.drop(columns=drop_columns, errors='ignore').copy()
X = X.fillna(X.median())

# Define models
models = [
    ('LinearRegression', LinearRegression()),
    ('ElasticNet', ElasticNet(random_state=42, max_iter=5000)),
    ('XGBoost', XGBRegressor(n_jobs=-1, random_state=42, verbosity=0)),
    ('CatBoost', CatBoostRegressor(verbose=0, random_state=42)),
    ('LightGBM', LGBMRegressor(n_jobs=-1, random_state=42, verbose=-1)),
    ('RandomForest', RandomForestRegressor(n_jobs=-1, random_state=42)),
    ('AdaBoost', AdaBoostRegressor(random_state=42)),
]

# K-Fold CV
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
results = {}

for name, model in models:
    print(f"Training {name}...")
    c_indices = []
    for train_idx, val_idx in kfold.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train_raw, y_val_raw = y_raw[train_idx], y_raw[val_idx]
        event_val = event[val_idx]
        
        # Transform y for training
        y_train_transformed = transform_y(y_train_raw)
        
        model.fit(X_train, y_train_transformed)
        pred_transformed = model.predict(X_val)
        
        # Apply CDF to get uniform scores (higher = shorter survival = higher risk)
        pred_uniform = inverse_transform_pred(pred_transformed)
        
        # C-index: higher pred_uniform should correspond to shorter survival
        # We use -pred_uniform as risk score (or pred_uniform directly if ranking inversely)
        c_idx = concordance_index(y_val_raw, -pred_uniform, event_val)
        c_indices.append(c_idx)
    
    results[name] = {'C-index': np.mean(c_indices), 'Std': np.std(c_indices)}
    print(f"  -> {name} C-index: {np.mean(c_indices):.4f} (+/- {np.std(c_indices):.4f})")

# Results
pd.DataFrame(results).T.sort_values(by='C-index', ascending=False)

Training LinearRegression...
  -> LinearRegression C-index: 0.7019 (+/- 0.0135)
Training ElasticNet...
  -> ElasticNet C-index: 0.6635 (+/- 0.0143)
Training XGBoost...
  -> XGBoost C-index: 0.6601 (+/- 0.0166)
Training CatBoost...
  -> CatBoost C-index: 0.7013 (+/- 0.0145)
Training LightGBM...
  -> LightGBM C-index: 0.6878 (+/- 0.0053)
Training RandomForest...
  -> RandomForest C-index: 0.7030 (+/- 0.0116)
Training AdaBoost...
  -> AdaBoost C-index: 0.6750 (+/- 0.0214)


,C-index,Std
RandomForest,0.703029,0.011559
LinearRegression,0.701860,0.013501
CatBoost,0.701341,0.014532
LightGBM,0.687756,0.005253
AdaBoost,0.674987,0.021406
ElasticNet,0.663523,0.014278
XGBoost,0.660086,0.016644


# Ensemble optim avec optuna

In [29]:
# import optuna
# from sklearn.model_selection import KFold, StratifiedKFold
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import roc_auc_score
# from sklearn.linear_model import LinearRegression, ElasticNet
# from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, AdaBoostRegressor
# from sklearn.svm import SVC
# from xgboost import XGBClassifier, XGBRegressor
# from catboost import CatBoostClassifier, CatBoostRegressor
# from lightgbm import LGBMClassifier, LGBMRegressor
# from lifelines.utils import concordance_index
# import numpy as np
# import pandas as pd

# optuna.logging.set_verbosity(optuna.logging.WARNING)

# # Prepare X
# X = clinical_preprocess_train.drop(columns=['ID', 'CENTER'], errors='ignore').copy()
# for col in X.select_dtypes(include=['object']).columns:
#     X[col] = LabelEncoder().fit_transform(X[col].astype(str))
# X = X.fillna(X.median())

# y_raw = targets_preprocess['OS_YEARS'].values
# event = event_target.values
# y_class = event.astype(int)

# # ============ CLASSIFICATION MODELS ============
# clf_models = {
#     'XGB': XGBClassifier(n_jobs=-1, random_state=42, verbosity=0),
#     'CatBoost': CatBoostClassifier(verbose=0, random_state=42),
#     'LGBM': LGBMClassifier(n_jobs=-1, random_state=42, verbose=-1),
#     'RF': RandomForestClassifier(n_jobs=-1, random_state=42),
#     'AdaBoost': AdaBoostClassifier(random_state=42, algorithm='SAMME'),
# }

# # ============ REGRESSION MODELS ============
# def transform_y(y):
#     n = len(y)
#     ranks = (-y).argsort().argsort() + 1
#     return norm.ppf(ranks / (n + 1))

# reg_models = {
#     'LinReg': LinearRegression(),
#     'ElasticNet': ElasticNet(random_state=42, max_iter=5000),
#     'XGB': XGBRegressor(n_jobs=-1, random_state=42, verbosity=0),
#     'CatBoost': CatBoostRegressor(verbose=0, random_state=42),
#     'LGBM': LGBMRegressor(n_jobs=-1, random_state=42, verbose=-1),
#     'RF': RandomForestRegressor(n_jobs=-1, random_state=42),
#     'AdaBoost': AdaBoostRegressor(random_state=42),
# }

# # ============ GET OOF PREDICTIONS ============
# def get_oof_preds_clf(models_dict, X, y, n_splits=5):
#     kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
#     oof = {name: np.zeros(len(y)) for name in models_dict}
#     for train_idx, val_idx in kf.split(X, y):
#         X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
#         y_tr = y[train_idx]
#         for name, model in models_dict.items():
#             m = model.__class__(**model.get_params())
#             m.fit(X_tr, y_tr)
#             oof[name][val_idx] = m.predict_proba(X_val)[:, 1]
#     return oof

# def get_oof_preds_reg(models_dict, X, y_raw, n_splits=5):
#     kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
#     oof = {name: np.zeros(len(y_raw)) for name in models_dict}
#     for train_idx, val_idx in kf.split(X):
#         X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
#         y_tr_transformed = transform_y(y_raw[train_idx])
#         for name, model in models_dict.items():
#             m = model.__class__(**model.get_params())
#             m.fit(X_tr, y_tr_transformed)
#             oof[name][val_idx] = m.predict(X_val)
#     return oof

# print("Getting OOF predictions for Classification...")
# oof_clf = get_oof_preds_clf(clf_models, X, y_class)

# print("Getting OOF predictions for Regression...")
# oof_reg = get_oof_preds_reg(reg_models, X, y_raw)

# # ============ OPTUNA BLEND OPTIMIZATION - CLASSIFICATION ============
# def objective_clf(trial):
#     weights = {name: trial.suggest_float(name, 0, 1) for name in oof_clf}
#     total = sum(weights.values())
#     blend = sum(oof_clf[name] * weights[name] / total for name in oof_clf)
#     return roc_auc_score(y_class, blend)

# print("\nOptimizing Classification Blend...")
# study_clf = optuna.create_study(direction='maximize')
# study_clf.optimize(objective_clf, n_trials=200, show_progress_bar=True)

# print(f"Best Classification AUC: {study_clf.best_value:.4f}")
# print(f"Best weights: {study_clf.best_params}")

# # ============ OPTUNA BLEND OPTIMIZATION - REGRESSION ============
# def objective_reg(trial):
#     weights = {name: trial.suggest_float(name, 0, 1) for name in oof_reg}
#     total = sum(weights.values())
#     blend = sum(oof_reg[name] * weights[name] / total for name in oof_reg)
#     pred_uniform = norm.cdf(blend)
#     return concordance_index(y_raw, -pred_uniform, event)

# print("\nOptimizing Regression Blend...")
# study_reg = optuna.create_study(direction='maximize')
# study_reg.optimize(objective_reg, n_trials=200, show_progress_bar=True)

# print(f"Best Regression C-index: {study_reg.best_value:.4f}")
# print(f"Best weights: {study_reg.best_params}")

# # ============ SUMMARY ============
# print("\n" + "="*50)
# print("CLASSIFICATION ENSEMBLE")
# print(f"  AUC: {study_clf.best_value:.4f}")
# for k, v in study_clf.best_params.items():
#     print(f"  {k}: {v:.3f}")

# print("\nREGRESSION ENSEMBLE")
# print(f"  C-index: {study_reg.best_value:.4f}")
# for k, v in study_reg.best_params.items():
#     print(f"  {k}: {v:.3f}")

# Final solution

On merge ensuite finalement les modèles ensembles pour obtenir un bon C index final selon la formule :

Risk = P(event = 0) * E[rank%|event = 0] + P(event = 1) * E[rank%| event = 1]

Cette formule nous dis :

Pour E[rank%| event = 1] on peut restreindre aux event = 1 ou on a un vrai classement,
Pour E[rank%| event = 0] o peut estimer les rangs parmis toutes les differentes positions puis 
se restreindre à ceux ci, en effet on a si T>T' et T' mort alors T mieux classer que T',
on a donc un ordre partiel dans nos données et on peut organiser de sorte a mettre entre les personne morte,
les personne encore en vie.

Une deuxieme approche est de faire du stacking en utilisant un meta modele, on peut aussi faire un nesemble entre le stacking et la meta modele

In [30]:
# =============================================================================
# OPTUNA UTILS - Avec IPCW C-index au lieu de concordance_index simple
# =============================================================================

import optuna
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, mean_squared_error
from sksurv.metrics import concordance_index_ipcw
from scipy.stats import norm
import numpy as np
import pandas as pd

optuna.logging.set_verbosity(optuna.logging.WARNING)

# --- Configuration ---
N_FOLDS = 5
RANDOM_STATE = 42


# --- Cross-Validation Factories ---
def get_stratified_kfold(n_splits=N_FOLDS):
    """Retourne un StratifiedKFold pour les problèmes de classification."""
    return StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)


def get_kfold(n_splits=N_FOLDS):
    """Retourne un KFold standard pour les problèmes de régression."""
    return KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)


# --- Transformation de Y ---
def transform_y(y):
    """Transforme y via rank -> uniform -> inverse CDF normale."""
    n = len(y)
    ranks = (-y).argsort().argsort() + 1  # Rang basé sur -temps
    uniform_scores = ranks / (n + 1)
    return norm.ppf(uniform_scores)


def inverse_transform_pred(pred):
    """CDF normale pour obtenir des scores uniformes."""
    return norm.cdf(pred)


# --- Conversion vers format sksurv ---
def make_survival_array(time, event):
    """Crée un structured array pour sksurv: dtype=[('event', bool), ('time', float)]"""
    return np.array(
        [(bool(e), t) for e, t in zip(event, time)],
        dtype=[("event", bool), ("time", float)],
    )


# --- Métriques ---
def compute_bic(n, k, nll_or_mse, is_regression=True):
    """Calcule le BIC. Pour régression: log(MSE), pour classification: NLL."""
    if is_regression:
        if nll_or_mse <= 0:
            nll_or_mse = 1e-10
        return k * np.log(n) + n * np.log(nll_or_mse)
    else:
        return k * np.log(n) + 2 * nll_or_mse


def compute_aic(k, nll):
    """Calcule l'AIC pour classification."""
    return 2 * k + 2 * nll


def safe_ipcw_cindex(
    y_train_time, y_train_event, y_test_time, y_test_event, pred, tau=None
):
    """
    IPCW C-index avec gestion d'erreurs.

    Args:
        y_train_time: Temps de survie sur train (pour estimer la censure)
        y_train_event: Événements sur train
        y_test_time: Temps de survie sur test
        y_test_event: Événements sur test
        pred: Prédictions de risque (plus élevé = plus de risque = survie plus courte)
        tau: Temps de troncature (optionnel, par défaut max du test)

    Returns:
        C-index IPCW
    """
    try:
        # Convertir en format sksurv
        survival_train = make_survival_array(y_train_time, y_train_event)
        survival_test = make_survival_array(y_test_time, y_test_event)

        # Tau = temps max observé dans le test si non spécifié
        if tau is None:
            tau = y_test_time.max()

        # IPCW C-index: estimate est la 2ème valeur retournée
        c_idx, _, _, _, _ = concordance_index_ipcw(
            survival_train, survival_test, pred, tau=tau
        )
        return c_idx
    except Exception as e:
        print(f"IPCW error: {e}")
        return 0.5  # Fallback

def safe_concordance_index(y_true, pred, event):
    """C-index simple (lifelines) avec gestion d'erreurs."""
    from lifelines.utils import concordance_index

    try:
        return concordance_index(y_true, pred, event)
    except ZeroDivisionError:
        return 0.5


# --- Comptage des feuilles pour modèles à arbres ---
def count_total_leaves(model):
    """Compte le nombre total de feuilles d'un modèle à arbres."""
    model_type = type(model).__name__

    if "XGB" in model_type:
        try:
            df = model.get_booster().trees_to_dataframe()
            return (df["Feature"] == "Leaf").sum()
        except:
            n_trees = model.n_estimators
            max_depth = getattr(model, "max_depth", 6)
            return n_trees * (2**max_depth)

    elif "LGBM" in model_type:
        try:
            booster = model.booster_
            return sum(booster.num_leaves())
        except:
            return model.n_estimators * model.num_leaves

    elif "CatBoost" in model_type:
        try:
            tree_count = model.tree_count_
            depth = model.get_param("depth") or 6
            return tree_count * (2**depth)
        except:
            return model.get_param("iterations") * (2 ** model.get_param("depth"))

    elif "RandomForest" in model_type:
        try:
            return sum(tree.get_n_leaves() for tree in model.estimators_)
        except:
            return model.n_estimators * (2 ** getattr(model, "max_depth", 10))

    elif "AdaBoost" in model_type:
        try:
            return sum(est.get_n_leaves() for est in model.estimators_)
        except:
            return model.n_estimators * 2

    else:
        return 0


print("Optuna utils with IPCW C-index loaded successfully!")


# =============================================================================
# EXEMPLE D'UTILISATION DANS evaluate_model
# =============================================================================
"""
def evaluate_model(model, X, y_raw, event):
    kfold = get_kfold()
    c_indices, mses = [], []
    
    for train_idx, val_idx in kfold.split(X):
        X_tr, X_val = X.iloc[train_idx].values, X.iloc[val_idx].values
        y_tr_t = transform_y(y_raw[train_idx])
        
        try:
            model.fit(X_tr, y_tr_t)
            pred = model.predict(X_val)
            
            # Utiliser IPCW C-index au lieu de concordance_index
            # pred doit être un score de risque: plus élevé = survie plus courte
            risk_score = norm.cdf(pred)  # Transformer en [0,1], score de risque
            
            c_idx = safe_ipcw_cindex(
                y_train_time=y_raw[train_idx],
                y_train_event=event[train_idx],
                y_test_time=y_raw[val_idx],
                y_test_event=event[val_idx],
                pred=risk_score  # Plus élevé = plus de risque
            )
            c_indices.append(c_idx)
            mses.append(np.mean((pred - transform_y(y_raw[val_idx]))**2))
        except Exception as e:
            c_indices.append(0.5)
            mses.append(1.0)
            
    return np.mean(c_indices), np.mean(mses)
"""


Optuna utils with IPCW C-index loaded successfully!


c:\Users\enzo.cAo\Documents\Projects\competitions\ens_data_challenge\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'\ndef evaluate_model(model, X, y_raw, event):\n    kfold = get_kfold()\n    c_indices, mses = [], []\n\n    for train_idx, val_idx in kfold.split(X):\n        X_tr, X_val = X.iloc[train_idx].values, X.iloc[val_idx].values\n        y_tr_t = transform_y(y_raw[train_idx])\n\n        try:\n            model.fit(X_tr, y_tr_t)\n            pred = model.predict(X_val)\n\n            # Utiliser IPCW C-index au lieu de concordance_index\n            # pred doit être un score de risque: plus élevé = survie plus courte\n            risk_score = norm.cdf(pred)  # Transformer en [0,1], score de risque\n\n            c_idx = safe_ipcw_cindex(\n                y_train_time=y_raw[train_idx],\n                y_train_event=event[train_idx],\n                y_test_time=y_raw[val_idx],\n                y_test_event=event[val_idx],\n                pred=risk_score  # Plus élevé = plus de risque\n            )\n            c_indices.append(c_idx)\n            mses.append(np.mean((pred - transform_y(y_raw

In [ ]:
# =============================================================================
# CELLULE: MODÈLES LINÉAIRES + SVM - Optimisation sur BIC
# =============================================================================
# Modèles Régression: Ridge, Lasso, ElasticNet, SVR (linear), SVR (RBF kernel)
# Modèles Classification: SVC (linear), SVC (RBF kernel)
# Objectif: Minimiser BIC = k * log(n) + n * log(MSE) pour régression
#                        = k * log(n) + 2 * NLL pour classification
# CV: 5-fold (StratifiedKFold pour classification)
# =============================================================================

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.svm import SVR, SVC
from sklearn.calibration import CalibratedClassifierCV

# --- Préparation des données ---
y_raw_linear = targets_preprocess["OS_YEARS"].values
event_linear = event_target.values
y_class_linear = event_linear.astype(int)
X_linear = (
    clinical_preprocess_train.drop(columns=drop_columns, errors="ignore")
    .copy()
    .fillna(0)
)
n_samples = len(y_raw_linear)


# --- Évaluation pour régression ---
def evaluate_linear_regression(model, X, y_raw, event):
    kfold = get_kfold()
    c_indices, mses = [], []

    for train_idx, val_idx in kfold.split(X):
        X_tr, X_val = X.iloc[train_idx].values, X.iloc[val_idx].values
        y_tr_transformed = transform_y(y_raw[train_idx])
        y_val_transformed = transform_y(y_raw[val_idx])

        try:
            model.fit(X_tr, y_tr_transformed)
            pred = model.predict(X_val)

            # MSE sur y transformé pour l'entraînement
            mse = mean_squared_error(y_val_transformed, pred)
            mses.append(mse)

            # C-index sur y original après retransformation
            pred_uniform = inverse_transform_pred(pred)
            c_idx = safe_concordance_index(
                y_raw[val_idx], -pred_uniform, event[val_idx]
            )
            c_indices.append(c_idx)
        except Exception as e:
            mses.append(1.0)
            c_indices.append(0.5)

    return np.mean(c_indices), np.mean(mses)


# --- Évaluation pour classification SVM (avec calibration pour probabilités) ---
def evaluate_svm_classification(model, X, y):
    kfold = get_stratified_kfold()
    aucs, nlls = [], []

    for train_idx, val_idx in kfold.split(X, y):
        X_tr, X_val = X.iloc[train_idx].values, X.iloc[val_idx].values
        y_tr, y_val = y[train_idx], y[val_idx]

        # Calibrer pour obtenir des probabilités
        calibrated_model = CalibratedClassifierCV(model, cv=3, method="sigmoid")
        calibrated_model.fit(X_tr, y_tr)
        y_proba = calibrated_model.predict_proba(X_val)[:, 1]

        aucs.append(roc_auc_score(y_val, y_proba))

        eps = 1e-10
        nll = -np.sum(
            y_val * np.log(y_proba + eps) + (1 - y_val) * np.log(1 - y_proba + eps)
        )
        nlls.append(nll)

    return np.mean(aucs), np.sum(nlls)


# --- Objectifs Optuna ---
def make_linear_objective(model_name, is_classification=False):
    def objective(trial):
        # ===== MODÈLES DE RÉGRESSION =====
        if model_name == "Ridge":
            model = Ridge(alpha=trial.suggest_float("alpha", 1e-4, 100, log=True))
            k = X_linear.shape[1]

        elif model_name == "Lasso":
            model = Lasso(
                alpha=trial.suggest_float("alpha", 1e-4, 10, log=True), max_iter=5000
            )
            k = X_linear.shape[1] // 2  # Estimation sparse

        elif model_name == "ElasticNet":
            model = ElasticNet(
                alpha=trial.suggest_float("alpha", 1e-4, 10, log=True),
                l1_ratio=trial.suggest_float("l1_ratio", 0.1, 0.9),
                max_iter=5000,
            )
            k = X_linear.shape[1] // 2

        elif model_name == "SVR_linear":
            model = SVR(
                kernel="linear",
                C=trial.suggest_float("C", 1e-3, 100, log=True),
                epsilon=trial.suggest_float("epsilon", 1e-4, 1, log=True),
            )
            k = X_linear.shape[1]

        elif model_name == "SVR_rbf":
            model = SVR(
                kernel="rbf",
                C=trial.suggest_float("C", 1e-3, 100, log=True),
                gamma=trial.suggest_float("gamma", 1e-5, 1, log=True),
                epsilon=trial.suggest_float("epsilon", 1e-4, 1, log=True),
            )
            # Pour RBF kernel, k est approximé par le nombre de support vectors
            k = X_linear.shape[1] * 2  # Approximation

        # ===== MODÈLES DE CLASSIFICATION =====
        elif model_name == "SVC_linear":
            model = SVC(
                kernel="linear",
                C=trial.suggest_float("C", 1e-3, 100, log=True),
                random_state=RANDOM_STATE,
            )
            k = X_linear.shape[1]

        elif model_name == "SVC_rbf":
            model = SVC(
                kernel="rbf",
                C=trial.suggest_float("C", 1e-3, 100, log=True),
                gamma=trial.suggest_float("gamma", 1e-5, 1, log=True),
                random_state=RANDOM_STATE,
            )
            k = X_linear.shape[1] * 2  # Approximation pour kernel

        # ===== ÉVALUATION =====
        if is_classification:
            auc, nll = evaluate_svm_classification(model, X_linear, y_class_linear)
            bic = compute_bic(n_samples, k, nll, is_regression=False)
            aic = compute_aic(k, nll)
            trial.set_user_attr("AUC", auc)
            trial.set_user_attr("AIC", aic)
        else:
            c_idx, mse = evaluate_linear_regression(
                model, X_linear, y_raw_linear, event_linear
            )
            bic = compute_bic(n_samples, k, mse, is_regression=True)
            trial.set_user_attr("C-index", c_idx)
            trial.set_user_attr("MSE", mse)

        trial.set_user_attr("BIC", bic)
        trial.set_user_attr("k", k)
        return bic  # Minimiser BIC

    return objective


# --- Exécution ---
linear_models_reg = ["Ridge", "Lasso", "ElasticNet",]
linear_models_clf = ["SVC_linear", "SVC_rbf"]

best_linear_results = {}

print("=== RÉGRESSION (modèles linéaires + SVR) ===")
for model_name in linear_models_reg:
    print(f"Optimizing {model_name}...")
    study = optuna.create_study(direction="minimize")
    study.optimize(
        make_linear_objective(model_name, is_classification=False),
        n_trials=50,
        show_progress_bar=True,
    )

    best_linear_results[model_name] = {
        "BIC": study.best_value,
        "C-index": study.best_trial.user_attrs["C-index"],
        "k": study.best_trial.user_attrs["k"],
        "params": dict(study.best_params),
    }
    print(
        f"  Best BIC: {study.best_value:.2f} | C-index: {study.best_trial.user_attrs['C-index']:.4f}"
    )

print("\n=== CLASSIFICATION (SVC) ===")
for model_name in linear_models_clf:
    print(f"Optimizing {model_name}...")
    study = optuna.create_study(direction="minimize")
    study.optimize(
        make_linear_objective(model_name, is_classification=True),
        n_trials=50,
        show_progress_bar=True,
    )

    best_linear_results[model_name] = {
        "BIC": study.best_value,
        "AUC": study.best_trial.user_attrs["AUC"],
        "k": study.best_trial.user_attrs["k"],
        "params": dict(study.best_params),
    }
    print(
        f"  Best BIC: {study.best_value:.2f} | AUC: {study.best_trial.user_attrs['AUC']:.4f}"
    )

# --- Résumé ---
print("\n" + "=" * 60)
print("RÉSUMÉ MODÈLES LINÉAIRES + SVM")
print(pd.DataFrame(best_linear_results).T)


In [ ]:

# %%
# =============================================================================
# CELLULE 3: MODÈLES À ARBRES - Optimisation avec régularisation sur |T|
# =============================================================================
# Modèles: XGBoost, CatBoost, LightGBM, RandomForest, AdaBoost
# Objectif: Minimiser (loss + alpha * |T|) où |T| = nombre total de feuilles
#           - Pour régression: loss = -C-index (on veut maximiser C-index)
#           - Pour classification: loss = -AUC
# CV: 5-fold (StratifiedKFold pour classification)
# =============================================================================

from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier

# ===============================
# PARAMÈTRE DE RÉGULARISATION
# ===============================
ALPHA_TREE_REG = (
    0.001  # Coefficient de régularisation sur |T| (à ajuster facilement ici)
)

RANDOM_STATE = 42

# --- Préparation des données ---
y_raw_tree = targets_preprocess["OS_YEARS"].values
event_tree = event_target.values
y_class_tree = event_tree.astype(int)
X_tree = (
    clinical_preprocess_train.drop(columns=drop_columns, errors="ignore")
    .copy()
    .fillna(0)
)


# --- Évaluation pour régression avec comptage des feuilles ---
def evaluate_tree_regression(model_class, params, X, y_raw, event):
    kfold = get_kfold()
    c_indices, total_leaves_list = [], []

    for train_idx, val_idx in kfold.split(X):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr_transformed = transform_y(y_raw[train_idx])

        try:
            model = model_class(**params)
            model.fit(X_tr, y_tr_transformed)
            pred = model.predict(X_val)

            # C-index sur y original après retransformation
            pred_uniform = inverse_transform_pred(pred)
            c_idx = safe_concordance_index(
                y_raw[val_idx], -pred_uniform, event[val_idx]
            )
            c_indices.append(c_idx)

            # Comptage des feuilles
            n_leaves = count_total_leaves(model)
            total_leaves_list.append(n_leaves)
        except Exception as e:
            c_indices.append(0.5)
            total_leaves_list.append(1000)

    return np.mean(c_indices), np.mean(total_leaves_list)


# --- Évaluation pour classification avec comptage des feuilles ---
def evaluate_tree_classification(model_class, params, X, y):
    kfold = get_stratified_kfold()
    aucs, total_leaves_list = [], []

    for train_idx, val_idx in kfold.split(X, y):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]

        try:
            model = model_class(**params)
            model.fit(X_tr, y_tr)
            y_proba = model.predict_proba(X_val)[:, 1]
            aucs.append(roc_auc_score(y_val, y_proba))

            n_leaves = count_total_leaves(model)
            total_leaves_list.append(n_leaves)
        except Exception as e:
            aucs.append(0.5)
            total_leaves_list.append(1000)

    return np.mean(aucs), np.mean(total_leaves_list)


# --- Factory pour les paramètres de chaque modèle ---
def suggest_tree_params(trial, model_name, is_classification=False):
    if "XGB" in model_name:
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 10, 200),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "n_jobs": -1,
            "verbosity": 0,
            "random_state": RANDOM_STATE,
        }
        model_class = XGBClassifier if is_classification else XGBRegressor

    elif "CatBoost" in model_name:
        params = {
            "iterations": trial.suggest_int("iterations", 10, 200),
            "depth": trial.suggest_int("depth", 4, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "verbose": 0,
            "random_state": RANDOM_STATE,
        }
        model_class = CatBoostClassifier if is_classification else CatBoostRegressor

    elif "LGBM" in model_name or "LightGBM" in model_name:
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 10, 200),
            "max_depth": trial.suggest_int("max_depth", 3, 15),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 20, 150),
            "n_jobs": -1,
            "verbose": -1,
            "random_state": RANDOM_STATE,
        }
        model_class = LGBMClassifier if is_classification else LGBMRegressor

    elif "RandomForest" in model_name:
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 10, 200),
            "max_depth": trial.suggest_int("max_depth", 5, 30),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "n_jobs": -1,
            "random_state": RANDOM_STATE,
        }
        model_class = (
            RandomForestClassifier if is_classification else RandomForestRegressor
        )

    elif "AdaBoost" in model_name:
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 10, 200),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 2.0, log=True),
            "random_state": RANDOM_STATE,
        }
        model_class = AdaBoostClassifier if is_classification else AdaBoostRegressor

    return model_class, params


# --- Objectif Optuna avec régularisation ---
def make_tree_objective(model_name, alpha=ALPHA_TREE_REG, is_classification=False):
    def objective(trial):
        model_class, params = suggest_tree_params(trial, model_name, is_classification)

        if is_classification:
            auc, n_leaves = evaluate_tree_classification(
                model_class, params, X_tree, y_class_tree
            )
            loss = -auc  # On veut maximiser AUC, donc minimiser -AUC
            trial.set_user_attr("AUC", auc)
        else:
            c_idx, n_leaves = evaluate_tree_regression(
                model_class, params, X_tree, y_raw_tree, event_tree
            )
            loss = -c_idx  # On veut maximiser C-index, donc minimiser -C-index
            trial.set_user_attr("C-index", c_idx)

        # Régularisation: loss + alpha * |T|
        regularized_loss = loss + alpha * n_leaves

        trial.set_user_attr("n_leaves", n_leaves)
        trial.set_user_attr("raw_loss", loss)
        trial.set_user_attr("regularized_loss", regularized_loss)

        return regularized_loss

    return objective


# --- Exécution ---
tree_models = ["XGBoost", "CatBoost", "LightGBM", "RandomForest", "AdaBoost"]

best_tree_results_reg = {}
best_tree_results_clf = {}

print(f"=== RÉGRESSION (modèles à arbres) | alpha={ALPHA_TREE_REG} ===")
for model_name in tree_models:
    print(f"Optimizing {model_name}...")
    study = optuna.create_study(direction="minimize")
    study.optimize(
        make_tree_objective(model_name, alpha=ALPHA_TREE_REG, is_classification=False),
        n_trials=50,
        show_progress_bar=True,
    )

    best_tree_results_reg[model_name] = {
        "C-index": study.best_trial.user_attrs["C-index"],
        "n_leaves": study.best_trial.user_attrs["n_leaves"],
        "reg_loss": study.best_value,
        "params": dict(study.best_params),
    }
    print(
        f"  C-index: {study.best_trial.user_attrs['C-index']:.4f} | "
        + f"Leaves: {study.best_trial.user_attrs['n_leaves']:.0f} | "
        + f"Reg.Loss: {study.best_value:.4f}"
    )

print(f"\n=== CLASSIFICATION (modèles à arbres) | alpha={ALPHA_TREE_REG} ===")
for model_name in tree_models:
    print(f"Optimizing {model_name}...")
    study = optuna.create_study(direction="minimize")
    study.optimize(
        make_tree_objective(model_name, alpha=ALPHA_TREE_REG, is_classification=True),
        n_trials=50,
        show_progress_bar=True,
    )

    best_tree_results_clf[model_name] = {
        "AUC": study.best_trial.user_attrs["AUC"],
        "n_leaves": study.best_trial.user_attrs["n_leaves"],
        "reg_loss": study.best_value,
        "params": dict(study.best_params),
    }
    print(
        f"  AUC: {study.best_trial.user_attrs['AUC']:.4f} | "
        + f"Leaves: {study.best_trial.user_attrs['n_leaves']:.0f} | "
        + f"Reg.Loss: {study.best_value:.4f}"
    )

# --- Résumé ---
print("\n" + "=" * 60)
print("RÉSUMÉ MODÈLES À ARBRES - RÉGRESSION")
print(pd.DataFrame(best_tree_results_reg).T[["C-index", "n_leaves", "reg_loss"]])

print("\nRÉSUMÉ MODÈLES À ARBRES - CLASSIFICATION")
print(pd.DataFrame(best_tree_results_clf).T[["AUC", "n_leaves", "reg_loss"]])



=== RÉGRESSION (modèles à arbres) | alpha=0.001 ===
Optimizing XGBoost...


  0%|          | 0/50 [00:00<?, ?it/s]

Best trial: 11. Best value: -0.590286: 100%|██████████| 50/50 [03:43<00:00,  4.46s/it]


  C-index: 0.6943 | Leaves: 104 | Reg.Loss: -0.5903
Optimizing CatBoost...


Best trial: 11. Best value: -0.544826: 100%|██████████| 50/50 [06:01<00:00,  7.23s/it]


  C-index: 0.7048 | Leaves: 160 | Reg.Loss: -0.5448
Optimizing LightGBM...


Best trial: 37. Best value: -0.36452: 100%|██████████| 50/50 [03:28<00:00,  4.17s/it]  


  C-index: 0.7145 | Leaves: 350 | Reg.Loss: -0.3645
Optimizing RandomForest...


Best trial: 11. Best value: -0.334479:  50%|█████     | 25/50 [01:29<00:42,  1.72s/it]